In [1]:
# | default_exp agent.planner.cache

In [1]:
# | export
import json
from pathlib import Path
from typing import Optional
from rich.console import Console

from agentic.agent.planner.models import ProjectContext, ProjectBreakdown

In [2]:
# | export

class CacheManager:
    """Handles execution state caching and restoration"""
    
    def __init__(self, console: Console, cache_dir: str = "execution_cache"):
        self.console = console
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.context_cache = self.cache_dir / "project_context.json"
        self.breakdown_cache = self.cache_dir / "project_breakdown.json"
    
    def save_to_cache(self, context: Optional[ProjectContext], breakdown: Optional[ProjectBreakdown]):
        """Save current execution state to cache"""
        try:
            if context:
                with open(self.context_cache, 'w') as f:
                    json.dump(context.model_dump(), f, indent=2, default=str)
            
            if breakdown:
                with open(self.breakdown_cache, 'w') as f:
                    json.dump(breakdown.model_dump(), f, indent=2)
                    
        except Exception as e:
            self.console.print(f"⚠️ Cache save failed: {e}")
    
    def restore_from_cache(self, user_request: str) -> tuple[Optional[ProjectContext], Optional[ProjectBreakdown]]:
        """Restore execution state from cache"""
        try:
            if not (self.context_cache.exists() and self.breakdown_cache.exists()):
                return None, None
            
            # Load context
            with open(self.context_cache, 'r') as f:
                context_data = json.load(f)
                
            # Check if it's the same request
            if context_data.get('original_request') != user_request:
                return None, None
                
            context = ProjectContext(**context_data)
            
            # Ensure current_artifacts is always a list (safety check)
            if not isinstance(context.current_artifacts, list):
                context.current_artifacts = []
            
            # Load breakdown
            with open(self.breakdown_cache, 'r') as f:
                breakdown_data = json.load(f)
            breakdown = ProjectBreakdown(**breakdown_data)
            
            return context, breakdown
            
        except Exception as e:
            self.console.print(f"⚠️ Cache restore failed: {e}")
            return None, None
    
    def clear_cache(self):
        """Clear cache files"""
        try:
            if self.context_cache.exists():
                self.context_cache.unlink()
            if self.breakdown_cache.exists():
                self.breakdown_cache.unlink()
        except Exception as e:
            self.console.print(f"⚠️ Cache clear failed: {e}")
